<a href="https://colab.research.google.com/github/ilmechaJu/-Base-Line-CNN/blob/main/ch05_CNN_%EA%B0%9C%EA%B3%A0%EC%96%91%EC%9D%B4_%EC%9D%B4%EB%AF%B8%EC%A7%80_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 7965854899137429876, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14509932544
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 5774420350831609736
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


import os

# 훈련 데이터는 Cats vs Dogs 데이터 셋을 사용합니다.
# 훈련 데이터의 다운로드와 압축해제는 아래 코드를 사용하면 됩니다.
!curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
!unzip -q kagglecatsanddogs_3367a.zip

#시험 데이터는 Oxford-IIIT Pet 데이터 세트를 사용합니다. 각 클래스에 대해 약 200개의 이미지가 포함된 37개의 애완 동물 데이터 세트입니다.
#시험 데이터의 다운로드와 압축해제는 아래 코드를 사용하면 됩니다.
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!tar -xzf images.tar.gz
!cd images ; mkdir img ; mv ./*jpg ./img

num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join("PetImages", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)




image_size = (150, 150)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

from keras.preprocessing.image import ImageDataGenerator

num_skipped = 0

folder_path = os.path.join("images", "img")
for fname in os.listdir(folder_path):
    fpath = os.path.join(folder_path, fname)
    try:
        fobj = open(fpath, "rb")
        is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
    finally:
        fobj.close()

    if not is_jfif:
        num_skipped += 1
        # Delete corrupted image
        os.remove(fpath)

print("Deleted %d images" % num_skipped)

text_ds = ImageDataGenerator(rescale=1./255)
# validation_datagen = ImageDataGenerator(rescale=1./255)

test_ds = text_ds.flow_from_directory(
    "./images",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)



data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = data_augmentation(inputs)

    # Entry block
    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape=image_size + (3,), num_classes=2)
keras.utils.plot_model(model, show_shapes=True)

epochs = 20

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)


y_pred = (np.expand_dims(test_ds.classes, axis=1).ravel()).astype(int)
# y_pred = (model.predict(text_ds) > 0.5).astype("int32")
np.savetxt('y_pred.csv', y_pred, fmt='%i')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  786M  100  786M    0     0   176M      0  0:00:04  0:00:04 --:--:--  181M
--2021-07-22 17:30:46--  https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 791918971 (755M) [application/x-gzip]
Saving to: ‘images.tar.gz’

images.tar.gz       100%[===================>] 755.23M  34.6MB/s    in 22s     

2021-07-22 17:31:09 (33.8 MB/s) - ‘images.tar.gz’ saved [791918971/791918971]

Deleted 1590 images
Found 23410 files belonging to 2 classes.
Using 18728 files for training.
Found 23410 files belonging to 2 classes.
Using 4682 files for validation.
Deleted 30 images
Found 7360 images belonging to 1 classes.
Epoch 1/20
5

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/20
586/586 [==============================] - 113s 192ms/step - loss: 0.4326 - accuracy: 0.8007 - val_loss: 0.4275 - val_accuracy: 0.8039
Epoch 3/20
586/586 [==============================] - 113s 192ms/step - loss: 0.3532 - accuracy: 0.8447 - val_loss: 0.4211 - val_accuracy: 0.8039
Epoch 4/20
586/586 [==============================] - 113s 192ms/step - loss: 0.3032 - accuracy: 0.8690 - val_loss: 0.3099 - val_accuracy: 0.8676
Epoch 5/20
586/586 [==============================] - 112s 191ms/step - loss: 0.2666 - accuracy: 0.8884 - val_loss: 0.3032 - val_accuracy: 0.8672
Epoch 6/20
586/586 [==============================] - 112s 190ms/step - loss: 0.2406 - accuracy: 0.8988 - val_loss: 0.2172 - val_accuracy: 0.9094
Epoch 7/20
586/586 [==============================] - 112s 190ms/step - loss: 0.2145 - accuracy: 0.9096 - val_loss: 0.2708 - val_accuracy: 0.8783
Epoch 8/20
586/586 [==============================] - 112s 190ms/step - loss: 0.1981 - accuracy: 0.9184 - val_loss: 0.4417 -